In [1]:
import numpy as np
from scipy.stats import chi2, kstest, norm
from scipy.optimize import minimize

In [2]:
X = np.array([5,8,6,12,14,18,11,6,13,7])
N = X.sum()

# Пункт a

#### $ \chi^2 $

In [9]:
delta = ((X-10)**2/10).sum()
pval = chi2.sf(delta, df=8)

print(f"delta: {delta:.4f}\npval:  {pval:.4f}")

delta: 16.4000
pval:  0.0370


#### Колмогоров

In [12]:
sup, pval = kstest(X, 'uniform', args=(0, 9))

print(f"sup*sqrtN: {sup:.4f}\npval:  {pval:.4f}")

sup*sqrtN: 0.5667
pval:  0.0015


# Пункт b

In [13]:
data = []
for i in range(len(X)):
    data.extend([i]*X[i])
initial_params = [np.mean(data), np.std(data)]
print(data, initial_params, sep = "\n")
data = np.array(data)

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9]
[np.float64(4.77), np.float64(2.505414137423193)]


#### $ \chi^2 $

In [6]:
def L(params):
    mu, sigma = params[0], params[1]
    if sigma <= 0:
        return np.inf
    return -np.sum(norm.logpdf(data, mu, sigma))

result = minimize(L, initial_params)

mu, sigma = result.x
print(f'Estimation of mu:\t{mu:.4f}')
print(f'Estimation of sigm:\t{sigma:.4f}')
print(f'Estimation of sigm^2:\t{sigma**2:.4f}')

Estimation of mu:	4.7700
Estimation of sigm:	2.5054
Estimation of sigm^2:	6.2771


In [14]:
bins = [-np.inf, 0, 1, 2, 3, 4, 5, 6, 7, 8, np.inf]
npi = []
for i in range(len(bins)-1):
    prob = norm.cdf(bins[i+1], mu, sigma) - norm.cdf(bins[i], mu, sigma)
    npi.append(prob * N)
npi = np.array(npi)
delta = np.sum((X - npi)**2 / npi)
pval = chi2.sf(delta, df=7)

print(f"delta: {delta:.4f}\npval:  {pval:.4f}")

delta: 14.3510
pval:  0.0453


#### Колмогоров

In [8]:
sup, pval = kstest(X, 'norm', args=(mu, sigma))

print(f"sup*sqrtN: {sup*N**0.5:.4f}\npval:  {pval:.4f}")

sup*sqrtN: 5.8826
pval:  0.0008
